In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
from fiona.crs import from_epsg
from shapely.geometry import Point, Polygon
import pysal as ps
import pylab as pl
%pylab inline
import seaborn
import re

Populating the interactive namespace from numpy and matplotlib


# 1. Playground: Data Cleaning

In [104]:
pg = gpd.read_file(filename='playground.shp')
print "# of all pg = {}".format(pg.shape[0])

# Delete the data with Accessible == N
pg = pg[pg.Accessible != 'N']
print "# of pg with Accessible == 'Y' = {}".format(pg.shape[0])

# keep the data with Status == 'open to the public' 
pg = pg[pg['Status'].apply(lambda x: x != None)]
pg.Status = map(str.lower, map(str, pg.Status))
pg = pg[pg.Status == 'open to the public']
print "# of pg with Accessible == 'Y' & Status =='open to the public' = {}".format(pg.shape[0])

# Reset index
pg = pg.reset_index(drop =True)

# pg with selected cols
pg = pg.loc[:,['Adaptive_S', 'Level', 'Location', 'Name', 'School_ID', 'geometry']]

# View
print "pg.crs = {}".format(pg.crs)
pg.head(2)

# of all pg = 1257
# of pg with Accessible == 'Y' = 892
# of pg with Accessible == 'Y' & Status =='open to the public' = 224
pg.crs = {'init': u'epsg:4326'}


,Adaptive_S,Level,Location,Name,School_ID,geometry
0,N,4,"43 Snyder Ave, Brooklyn, NY 11226",PS 6K,K006,POINT (-73.95659999999999 40.6489)
1,N,4,"80 Underhill Avenue, Brooklyn, NY 11238",PS 9K,K009,POINT (-73.9658 40.6785)


In [105]:
pg.to_file('pg')

<br/>
# 2. Restaurant: Data Cleaning

In [182]:
# rr for restaurant
rr = pd.read_csv('NYCRestaurant.csv', header=None)
rr = rr.iloc[:, [0,1,2]]
rr = rr.rename(columns={0: 'info', 1: 'lat', 2: 'lon'})

# geometry
rr['lonlat'] = zip(rr.lon, rr.lat)
rr['geometry'] = rr[['lonlat']].applymap(lambda x: Point(x))
rr = rr.loc[:, ['info', 'geometry']]

# to GeoDataFrame
rr = gpd.GeoDataFrame(rr)
rr.crs = pg.crs
rr.to_file("rr")